In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import struct

In [ ]:
NAME = 'aneto'

In [ ]:
def readAscFile(filename):
    
    f = open(filename, 'r')
    
    ncols = nrows = 0
    xmin = ymin = 0.0
    cellSize = 0.0
    centered = False
    nodata = -999
    
    lenHeader = 6
    for i in range(lenHeader):
        line = f.readline()
        vals = line.split()
        key = vals[0].lower()
        
        if key == 'ncols':
            ncols = int(vals[1])
        if key == 'nrows':
            nrows = int(vals[1])
        if key == 'xllcenter':
            xmin = float(vals[1])
            centered = True
        if key == 'yllcenter':
            ymin = float(vals[1])
            centered = True
        if key == 'xllcorner':
            xmin = float(vals[1])
        if key == 'yllcorner':
            ymin = float(vals[1])
        if key == 'cellsize':
            cellSize = float(vals[1])
        if 'nodata' in key:
            nodata = float(vals[1])
    if centered:
        xmin -= 0.5*cellSize
        ymin -= 0.5*cellSize
        
    # read elevation values
    H = np.zeros((ncols*nrows, ), dtype=float)
    idx = 0
    for line in f:
        strings = line.split()
        for str in strings:
            H[idx] = float(str)
            idx += 1
    H = H.reshape((nrows, ncols))
            
    return H, nodata, cellSize, (nrows, ncols), (xmin, ymin)

In [ ]:
H, nodata, resolution, points, pmin = readAscFile(NAME + '.asc')
nrows, ncols = points

In [ ]:
print('dimensions', H.shape)
print('resolution', resolution)
print('heights', H.min(), H.max())

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(H, interpolation='nearest')

In [ ]:
fply = open(NAME + '.ply', 'wb')
fply.write(bytes('ply\n', encoding='ascii'))
fply.write(bytes('format binary_little_endian 1.0\n', encoding='ascii'))
fply.write(bytes('element vertex %d\n' % (nrows*ncols), encoding='ascii'))
fply.write(bytes('property float x\n', encoding='ascii'))
fply.write(bytes('property float y\n', encoding='ascii'))
fply.write(bytes('property float z\n', encoding='ascii'))
fply.write(bytes('element face %d\n' % ((nrows-1)*(ncols-1)*2), encoding='ascii'))
fply.write(bytes('property list uint8 int32 vertex_index\n', encoding='ascii'))
fply.write(bytes('end_header\n', encoding='ascii'))       
for i in range(nrows):
    for j in range(ncols):
        fply.write(struct.pack('<fff', pmin[0] + j*resolution, pmin[1] + (nrows-1-i)*resolution, H[i][j]))

for i in range(nrows-1):
    for j in range(ncols-1):                 
        fply.write(struct.pack('<Biii', 3, i*ncols + j, (i+1)*ncols + j, i*ncols + j + 1))
        fply.write(struct.pack('<Biii', 3, (i+1)*ncols + j, (i+1)*ncols + j + 1, i*ncols + j + 1))
fply.close()  